### Semantic kernel Vector Search with chat context/history 

In previous session, we walked through a simple example based on company ABC data. In that example, and employee wanted to simply connect to company's data, retrieve some info, and add a new record.

Now let's consider a different scenario. 

Let's assume the company has a chatbot that provides support to customers. For an optimal experience, the chatbot needs to not only access to all product, sales, shipment and return data, but also it would need to be able to pick up the conversation with each customer where it was left off last. 

For this usecase, we are going to introduce and use below semantic kernel capabilities (in addition to what we covered before) to develop a solution:
- Chat completion **Agent** to manage the conversation and data access (one agent)
  -- example: we don't want the agent to access data for other customers or sensitive company data. 
- Vector search using Semantic Kernel Vector Store connectors
  -- Use company data as context to guide the conversation aka **RAG**
- Store and maintain **chat history** per customer and retrieve the history when needed.
  -- We will persist the history in CosmosDB



In [ ]:
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
# importing all we need for this session
from semantic_kernel.kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from typing import TypedDict, Annotated, List, Optional
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)
from semantic_kernel.contents.chat_history import ChatHistory

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)


kernel = Kernel()

# Auto-loads defaults from .env file, alternatively you can set endpoint, deployment_name and api_key directly
chat_completion = AzureChatCompletion()

kernel.add_service(chat_completion)

In [2]:
import src.pg_data_plugin


Connection uri was rertieved successfully.
